In [ ]:
# default_exp density_ratio_em

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# export
from multiinstance.em import generateBags
from multiinstance.utils import estimate
from multiinstance.nnpu import getPosterior as getNNPUPosterior
from multiinstance.vpu import train as getVPUPosterior
from multiinstance.ranker import RankNet

import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as ss
from sklearn.cluster import KMeans
from sklearn.metrics import roc_auc_score
from easydict import EasyDict
from scipy.spatial.distance import cdist
from tqdm.notebook import tqdm

from multiinstance.data.gaussian_dg import GaussianMixtureDataGenerator

from dist_curve.transforms import getOptimalTransform

from multiinstance.ranker import RankNet

from tqdm.notebook import trange

In [ ]:
# export
class DensityRatioEM:
    def __init__(self, bags,n_clusters):
        self.bags = bags
        self.n_clusters = n_clusters
        self.findGlobalClusters()
        self.tau_posS = []
        self.tau_uS = []
        self.debugLabels= []
        self.debugPosteriors = []
        self.clusterAlphaHats = np.zeros(n_clusters)
        for cnum in trange(n_clusters,desc="transforms and class prior estimation"):
            self.estimateClusterClassPrior(cnum)
            
    
    def findGlobalClusters(self):
        "Run K-Means on the positives from all bags then assign each unlabeled point to a cluster based on the resulting clusters of K-Means"
        globalPositives = np.concatenate([b.X_pos for b in bags])
        kmeans = KMeans(n_clusters=self.n_clusters).fit(globalPositives)
        # Cluster Unlabeled
        for bagNum,b in enumerate(self.bags):
            self.bags[bagNum].unlabeled_cluster_assignment = kmeans.predict(b.x_unlabeled)
            self.bags[bagNum].positive_cluster_assignment = kmeans.predict(b.X_pos)
        self.kmeans = kmeans
    
    def getClusterMembers(self,cnum):
        "Get the unlabeled and positive points across all bags that were assigned the given cluster"
        unlabeledInCluster = [b.x_unlabeled[b.unlabeled_cluster_assignment == cnum] for b in self.bags]
        posInCluster = [b.X_pos[b.positive_cluster_assignment == cnum] for b in self.bags]
        unlabeled = np.concatenate(unlabeledInCluster)
        positive = np.concatenate(posInCluster)
        return unlabeled, positive
    
    def trainTransform(self,cnum):
        "Train and store the univariate PU transform for a given cluster"
        unlabeled,positive = self.getClusterMembers(cnum)
        x = np.concatenate((positive,unlabeled))
        y = np.concatenate((np.ones(positive.shape[0]),
                            np.zeros(unlabeled.shape[0])))
        tau,aucpu = getOptimalTransform(x,y)
        tau_pos = np.ascontiguousarray(tau[:positive.shape[0]].reshape((-1,1)))
        tau_u = np.ascontiguousarray(tau[positive.shape[0]:].reshape((-1,1)))
        self.tau_posS.append(tau_pos)
        self.tau_uS.append(tau_u)

    def estimateClusterClassPrior(self,cnum):
        "Estimate the class prior for the given cluster"
        self.trainTransform(cnum)
        p,u = self.tau_posS[cnum], self.tau_uS[cnum]
        self.clusterAlphaHats[cnum],_ = estimate(p,u)

    def ratioFromPosteriorVec(self, posts, alpha):
        "convert posterior vector and class prior estimate for a cluster into a density ratio vector"
        return np.nan_to_num((alpha * (1 - posts)) / (posts * (1 - alpha)))
    
    def trueDensityRatio(self,cnum):
        """
        Get the true density ratio
        
        Arguments:
        - cnum : int : cluster number
        """
        # see formatComponentInfo
        comp = self.components[cnum]
        unlabeled,_ = self.getClusterMembers(cnum)
        # Positive density
        f1 = ss.multivariate_normal.pdf(unlabeled,mean=comp.posMean,cov=comp.posCov)
        # Negative density
        f0 = ss.multivariate_normal.pdf(unlabeled,mean=comp.negMean,cov=comp.negCov)
#         ratios = f0/f1
        alpha = self.clusterAlphaHats[cnum]
        posts = f1 * alpha / (f1 * alpha + (1-alpha) * f0)
        ratios = self.ratioFromPosteriorVec(posts,alpha)
        return ratios

    def trainPNPosterior(self,xPN,yPN):
        "get estimated pn posterior probabilities for given data"
        posteriors,auc = getOptimalTransform(xPN,yPN)
        return posteriors
    
    def densityRatioFromEstimatedPNPosterior(self,cnum):
        "For a given cluster use the true PN labels to train a classifier modeling the PN posterior then calculate density ratio"
        unlabeled,positive = self.getClusterMembers(cnum)
        hiddenLabels = np.concatenate([b.hiddenLabels[b.unlabeled_cluster_assignment == cnum] for b in self.bags])
        posteriors = self.trainPNPosterior(unlabeled,hiddenLabels.astype(bool))
        self.debugLabels.append(hiddenLabels)
        self.debugPosteriors.append(posteriors)
        print("posterior auc ",roc_auc_score(hiddenLabels,posteriors))
        ratios = self.ratioFromPosteriorVec(posteriors,self.clusterAlphaHats[cnum])
        return ratios
        """
        xPN = np.concatenate((positive,unlabeled))
        yPN = np.concatenate((np.ones(positive.shape[0]),hiddenLabels)).astype(bool)
        posteriors = self.trainPNPosterior(xPN,yPN)[positive.shape[0]:]
        print("posteriors auc",roc_auc_score(hiddenLabels, posteriors))
        NP,NU = positive.shape[0], unlabeled.shape[0]
        classPrior = (NP + self.clusterAlphaHats[cnum] * NU) / (NP + NU)
        ratios = self.ratioFromPosteriorVec(posteriors, classPrior)
        return ratios
        """
        

    def getNNPUInput(self,cnum,inputType):
        """
        Get the input to AbsNNPU
        
        Arguments:
        - cnum : int : cluster number
        - inputType : {raw,transform,estimated rank,true rank} : space in which AbsNNPU will be trained

        Return Values:
        - xPU : ndarray (n,d) : instances in feature space
        - yPU : ndarray (n,) : PU labels
        - inputs : ndarray (n,k) : instances in space AbsNNPU will be trained
        """
        assert inputType in "raw,transform,estimated rank,true rank".split(",")
        unlabeled,positive = self.getClusterMembers(cnum)
        num_pos = positive.shape[0]
        xPU = np.concatenate((positive,unlabeled))
        yPU = np.concatenate((np.ones(positive.shape[0]), np.zeros(unlabeled.shape[0])))[:,None]
        if inputType == "raw":
            return xPU,yPU,xPU,num_pos
        elif inputType == "transform":
            tauPos, tauU = self.tau_posS[cnum], self.tau_uS[cnum]
            transforms = np.concatenate((tauPos, tauU))
            return xPU,yPU,transforms,num_pos
        elif inputType == "estimated rank":
            # Run RankNet or xgboost
            ranknet = RankNet()
            ranks = ranknet.train(xPU,yPU.ravel().astype(bool))[:,None]
            print("estimated ranks aucpu",roc_auc_score(yPU,ranks))
            return xPU,yPU,ranks,num_pos
        else:
            # True Rank
            puposteriorFn = self.components[cnum].dg.pu_posterior
            ranks = ss.rankdata(puposteriorFn(xPU - self.components[cnum].r))[:,None]
            ranks = (ranks - ranks.min()) / (ranks.max() - ranks.min())
            return xPU,yPU,ranks,num_pos

    def estimatePosteriors(self,x,y,inputs,cnum):
        "Return the estimated pn posteriors"
        posteriors,net = getNNPUPosterior(x.astype(np.float32),
                                          y,
                                          self.clusterAlphaHats[cnum],
                                          inputs.astype(np.float32),
                                          training_args=EasyDict({'n_units': 1000,
                                                         'n_hidden': 10,
                                                         'dropout_rate': 0.5,
                                                         'maxIter': 500,
                                                         'batch_size': 128}))
        return posteriors

    def estimateClusterDensityRatio(self,cnum,inputType):
        x,y,inputs,num_pos = self.getNNPUInput(cnum,inputType)
        posteriors = self.estimatePosteriors(x,y,inputs,cnum)
        posteriors = posteriors[num_pos:]
        ratios = self.ratioFromPosteriorVec(posteriors,self.clusterAlphaHats[cnum])
        return ratios

    def splitRatiosIntoBags(self,ratios,cnum):
        "given list of density ratios for all unlabeled points, separate back into lists for each bag"
        bagRatios = []
        numUnlabeled = [b.x_unlabeled[b.unlabeled_cluster_assignment == cnum].shape[0] for b in self.bags]
        endPoints = np.concatenate(([0],np.cumsum(numUnlabeled)))
        for bagNum in range(1, len(endPoints)):
            r = ratios[endPoints[bagNum - 1]:endPoints[bagNum]]
            bagRatios.append(r)
        return bagRatios
    
    def formatComponentInfo(self,componentInfo):
        """
        The components found by kmeans likely don't match the true order of the components,
        find the mapping and create EasyDicts for each comp
        
        Arguments:
        - componentInfo EasyDict with fields:
            - posMeans : length <numComps> list of (<dim>) ndarrays
            - negMeans : length <numComps> list of (<dim>) ndarrays
            - posCovs  : length <numComps> list of (<dim>,<dim>) ndarrays
            - negCovs  : length <numComps> list of (<dim>,<dim>) ndarrays
            - dgs      : list of GaussianMixtureDataGenerator objects (see multiinstance.data.gaussian_dg)
        """
        if componentInfo is None:
            self.components = None
            return
        # maps kmeans cluster to true component index
        clusterMap = cdist(self.kmeans.cluster_centers_, componentInfo.posMeans).argmin(1)
        self.components = []
        for comp in clusterMap:
            compInfo = EasyDict()
            compInfo.posMean = componentInfo.posMeans[comp]
            compInfo.negMean = componentInfo.negMeans[comp]
            compInfo.posCov = componentInfo.posCovs[comp]
            compInfo.negCov = componentInfo.negCovs[comp]
            compInfo.dg = componentInfo.dgs[comp]
            compInfo.r = componentInfo.rs[comp]
            self.components.append(compInfo)
        
    def getBagRatios(self,inputType,componentInfo=None):
        """
        Arguments:
        - inputType : {true posterior,
                       estimated posterior,
                       nnpu raw,
                       nnpu transform,
                       nnpu estimated rank,
                       nnpu true rank}
        - componentInfo : Required if inputType in {"true posterior","nnpu true rank"}
            - See self.formatComponentInfo for details
            
        """
        # NClusters x NBags size list containing the density ratio for the unlabeled points
        # from the specified bag in the specified cluster
        self.bagRatios = []
        self.formatComponentInfo(componentInfo)
        for cnum in trange(self.n_clusters):
            if inputType == "true posterior":
                ratios = self.trueDensityRatio(cnum)
            elif inputType == "estimated posterior":
                ratios = self.densityRatioFromEstimatedPNPosterior(cnum)
            elif "nnpu" in inputType:
                ratios = self.estimateClusterDensityRatio(cnum,inputType.replace("nnpu ",""))
            else:
                raise ValueError("Invalid inputType specified: {}".format(inputType))
            self.bagRatios.append(self.splitRatiosIntoBags(ratios,cnum))
        if inputType == "estimated posterior":
            print(roc_auc_score(np.concatenate(self.debugLabels),
                                np.concatenate(self.debugPosteriors)))
    def EM(self,NIters=500):
        self.eta = np.zeros((len(self.bags), self.n_clusters))
        for cnum in range(self.n_clusters):
            for bagNum, b in enumerate(self.bags):
                ratios = self.bagRatios[cnum][bagNum]
                eta_i_j = np.array(.5)
                for em_iter in range(NIters):
                    den = eta_i_j + (1 - eta_i_j) * ratios
                    eta_i_j = np.mean(eta_i_j / den)
                self.eta[bagNum,cnum] = eta_i_j
    
    def run(self,densityRatioInputType,componentInfo=None):
        self.getBagRatios(densityRatioInputType, componentInfo=componentInfo)
        self.EM()
        self.estimateBagParameters()
    
    
    def posterior(self, bagNum, clusterNum):
        eta_i_j = self.eta[bagNum, clusterNum]
        densityRatios = self.bagRatios[clusterNum][bagNum]
        return eta_i_j / (eta_i_j + (1 - eta_i_j)*densityRatios)
    
    def getAUC(self):
        labels = []
        posts = []
        for bagNum in range(len(self.bags)):
            for cnum in range(self.n_clusters):
                posts.append(self.posterior(bagNum,cnum))
                labels.append(self.bags[bagNum].hiddenLabels[self.bags[bagNum].unlabeled_cluster_assignment == cnum])
        labels = np.concatenate(labels)
        posts = np.concatenate(posts)
        return roc_auc_score(labels, posts)

    def estimateBagParameters(self):
        N = len(self.bags)
        self.alphaHats = np.zeros(N)
        self.pi = np.zeros((N,self.n_clusters))
        self.rho = np.zeros((N,self.n_clusters))
        for bagNum, b in enumerate(self.bags):
            eta_j = self.eta[bagNum]
            gamma_j = np.unique(b.unlabeled_cluster_assignment,
                                return_counts=True)[1] / b.unlabeled_cluster_assignment.shape[0]
            alpha_j = eta_j.dot(gamma_j)
            pi_j = np.multiply(eta_j, gamma_j) / alpha_j
            rho_j = np.multiply(1 - eta_j, gamma_j) / (1 - alpha_j)
            self.alphaHats[bagNum] = alpha_j
            self.pi[bagNum] = pi_j
            self.rho[bagNum] = rho_j

# Baseline

In [ ]:
def posteriorCorrection(tau, alpha, S0S1):
    post =  alpha * S0S1 * (tau / (1 - tau))
    post = np.nan_to_num(post)
    return post

def baseline(bags,pupost=None):
    # apply transforms
    posts = []
    labels = []
    absErrs = []
    for b in tqdm(bags,leave=False):
        tau, aucpu = getOptimalTransform(np.concatenate((b.X_pos,
                                                         b.x_unlabeled)), 
                                         np.concatenate((np.ones(b.X_pos.shape[0]),
                                                        np.zeros(b.x_unlabeled.shape[0]))))
        tau_pos = np.ascontiguousarray(tau[:b.X_pos.shape[0]].reshape((-1,1)))
        tau_u = np.ascontiguousarray(tau[b.X_pos.shape[0]:].reshape((-1,1)))
        alphaHat,_ = estimate(tau_pos, tau_u)
        alphaHat = alphaHat.numpy()[0,0]
        absErrs.append(np.abs(alphaHat - b.alpha))
        s0s1 = b.x_unlabeled.shape[0] / b.X_pos.shape[0]
        p,net = getNNPUPosterior(np.concatenate((b.X_pos,b.x_unlabeled)).astype(np.float32),
                                     np.concatenate((np.ones((tau_pos.shape[0],1)),
                                                     np.zeros((tau_u.shape[0],1)))).astype(np.int32),
                                    alphaHat,pupost=pupost)
        posts.append(p[tau_pos.shape[0]:])
        labels.append(b.hiddenLabels)
        print(roc_auc_score(np.concatenate(labels),
                         np.concatenate(posts)))
    return roc_auc_score(np.concatenate(labels),
                         np.concatenate(posts)), np.mean(absErrs)

In [ ]:
def generateParams(nClusters, nComps, dim,aucpn_range=[0.8,0.85]):
    posMeans,posCovs,negMeans,negCovs,dgs,rs =[], [],[],[],[],[]
    for comp in range(nClusters):
        dg = GaussianMixtureDataGenerator(dim,nComps,aucpn_range,100,1000)
        pos,neg = dg.components_pos[0], dg.components_neg[0]
        r = np.zeros(dim)
        for _ in range(1):
            r += np.random.normal(loc=0,scale=10,size=dim)
        posMeans.append(pos.mean+r)
        posCovs.append(pos.cov)
        negMeans.append(neg.mean+r)
        negCovs.append(neg.cov)
        dgs.append(dg)
        rs.append(r)
    return EasyDict({"posMeans":posMeans,
                     "posCovs": posCovs,
                     "negMeans": negMeans,
                     "negCovs": negCovs,
                     "dgs":dgs,
                     "rs": rs})

# Test

In [ ]:
NClusters = 3
Dim = 8
aucpn_range=  [.8,1.]
NBags = 1000
NPos = 10
NUnlabeled = 25

In [ ]:
componentInfo = generateParams(NClusters,1,Dim,aucpn_range=aucpn_range)

In [ ]:
bags = generateBags(NBags,
                    pos_means=componentInfo.posMeans,
                    neg_means=componentInfo.negMeans,
                    pos_covs=componentInfo.posCovs,
                    neg_covs=componentInfo.negCovs,
                    NPos=NPos,NUnlabeled=NUnlabeled)

In [ ]:
drem = DensityRatioEM(bags,NClusters)

In [ ]:
drem.run("true posterior",componentInfo)

/home/dz/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:273: RuntimeWarning: invalid value encountered in true_divide


In [ ]:
drem.getAUC()

0.7550034674351558

In [ ]:
drem.run("estimated posterior",)

posterior auc  0.7580367534545117


posterior auc  0.7589458741189482


/home/dz/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:54: RuntimeWarning: divide by zero encountered in true_divide


posterior auc  0.7573873849037237

0.758215097741042


/home/dz/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:273: RuntimeWarning: invalid value encountered in true_divide
/home/dz/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:273: RuntimeWarning: divide by zero encountered in true_divide


In [ ]:
drem.getAUC()

0.6373517985082932

In [ ]:
drem.run("nnpu raw")

In [ ]:
drem.getAUC()

0.6355083784146899

In [ ]:
drem.run("nnpu transform")

In [ ]:
drem.getAUC()

0.6421627426263373

In [ ]:
drem.run("nnpu estimated rank")

37/37 [==============================] - 0s 1ms/step - loss: 0.6899
minvalLoss,  0.6898773908615112
37/37 [==============================] - 0s 750us/step - loss: 0.6897
minvalLoss,  0.6896600127220154
37/37 [==============================] - 0s 1ms/step - loss: 0.6869
minvalLoss,  0.6868950128555298
37/37 [==============================] - 0s 966us/step - loss: 0.6865
minvalLoss,  0.6865429282188416


37/37 [==============================] - 0s 883us/step - loss: 0.7001
minvalLoss,  0.7000758051872253
37/37 [==============================] - 0s 1ms/step - loss: 0.6915
minvalLoss,  0.6915297508239746
37/37 [==============================] - 0s 1ms/step - loss: 0.6819
minvalLoss,  0.6819350123405457
37/37 [==============================] - 0s 730us/step - loss: 0.7015


37/37 [==============================] - 0s 1ms/step - loss: 0.7091
minvalLoss,  0.709122359752655
37/37 [==============================] - 0s 962us/step - loss: 0.6945
minvalLoss,  0.6945189833641052
37/37 [==============================] - 0s 758us/step - loss: 0.6849
minvalLoss,  0.6848810911178589
37/37 [==============================] - 0s 810us/step - loss: 0.7194


37/37 [==============================] - 0s 908us/step - loss: 0.7106
minvalLoss,  0.7106239199638367
37/37 [==============================] - 0s 765us/step - loss: 0.6869
minvalLoss,  0.6868681907653809
37/37 [==============================] - 0s 710us/step - loss: 0.6817
minvalLoss,  0.6816834211349487
37/37 [==============================] - 0s 845us/step - loss: 0.7087


37/37 [==============================] - 0s 1ms/step - loss: 0.6963
minvalLoss,  0.6962676644325256
37/37 [==============================] - 0s 837us/step - loss: 0.6962
minvalLoss,  0.6961689591407776
37/37 [==============================] - 0s 830us/step - loss: 0.6896
minvalLoss,  0.689607560634613
37/37 [==============================] - 0s 724us/step - loss: 0.6878
minvalLoss,  0.6878001093864441
37/37 [==============================] - 0s 740us/step - loss: 0.7183
0.6009685405514837
estimated ranks aucpu 0.6009685405514837


...

37/37 [==============================] - 0s 1ms/step - loss: 0.7029
minvalLoss,  0.7028703093528748
37/37 [==============================] - 0s 855us/step - loss: 0.6896
minvalLoss,  0.6896032691001892
37/37 [==============================] - 0s 1ms/step - loss: 0.6862
minvalLoss,  0.6861523985862732
37/37 [==============================] - 0s 1ms/step - loss: 0.6840
minvalLoss,  0.6839578747749329
37/37 [==============================] - 0s 1ms/step - loss: 0.7203


37/37 [==============================] - 0s 848us/step - loss: 0.6968
minvalLoss,  0.6968413591384888
37/37 [==============================] - 0s 1ms/step - loss: 0.6858
minvalLoss,  0.6858195066452026
37/37 [==============================] - 0s 727us/step - loss: 0.6782
minvalLoss,  0.6781830191612244
37/37 [==============================] - 0s 729us/step - loss: 0.6922


37/37 [==============================] - 0s 1ms/step - loss: 0.6998
minvalLoss,  0.6997700929641724
37/37 [==============================] - 0s 1ms/step - loss: 0.6964
minvalLoss,  0.6963723301887512
37/37 [==============================] - 0s 1ms/step - loss: 0.6909
minvalLoss,  0.6909434199333191
37/37 [==============================] - 0s 1ms/step - loss: 0.6886
minvalLoss,  0.6886197924613953
37/37 [==============================] - 0s 864us/step - loss: 0.6880
minvalLoss,  0.6879541873931885
37/37 [==============================] - 0s 760us/step - loss: 0.6869
minvalLoss,  0.6869078874588013
37/37 [==============================] - 0s 2ms/step - loss: 0.7078


37/37 [==============================] - 0s 2ms/step - loss: 0.7046
minvalLoss,  0.7046396136283875
37/37 [==============================] - 0s 955us/step - loss: 0.6918
minvalLoss,  0.6918184161186218
37/37 [==============================] - 0s 917us/step - loss: 0.6899
minvalLoss,  0.6898775100708008
37/37 [==============================] - 0s 790us/step - loss: 0.6834
minvalLoss,  0.6833839416503906
37/37 [==============================] - 0s 762us/step - loss: 0.7064


37/37 [==============================] - 0s 1ms/step - loss: 0.7074
minvalLoss,  0.707352340221405
37/37 [==============================] - 0s 1ms/step - loss: 0.6923
minvalLoss,  0.6923142075538635
37/37 [==============================] - 0s 1ms/step - loss: 0.6886
minvalLoss,  0.6886152029037476
37/37 [==============================] - 0s 712us/step - loss: 0.7000
0.574532188525778
estimated ranks aucpu 0.574532188525778


In [ ]:
drem.getAUC()

0.5720064601987372

In [ ]:
drem.run("nnpu true rank",componentInfo=componentInfo)

/home/dz/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:272: RuntimeWarning: invalid value encountered in true_divide


In [ ]:
drem.getAUC()

0.7140038143549028

In [ ]:
baseline(bags)

0.5595238095238095


0.564236111111111


0.5058905058905059


0.5766509433962264


0.6133333333333334


...

0.5144298466480514


0.5142221423699115


0.514567881435321


0.5151236773161852


0.514753776903848


# Experiment 1
NClusters = 3
Dim = 2
aucpn_range=  [.8,1.]
NBags = 250
NPos = 10
NUnlabeled = 25


* true posterior : 0.747
* nnpu transform : 0.708
* nnpu raw : 0.692
* nnpu true rank : 0.66
* baseline : 0.578
* nnpu est rank : 0.575
* Estimated posterior : 0.54

nnpu raw outperforms the baseline when there is a large number of small bags. This is the expected result as the class prior and posterior estimates are difficult in the small data setting.

# Experiment 2
NClusters = 1
Dim = 2
aucpn_range=  [.8,1.]
NBags = 1
NPos = 500
NUnlabeled = 1000

* NNPU Estimated Rank : 0.935
* True Posterior : 0.898
* NNPU True Rank : 0.897
* Estimated Posterior : 0.862
* Baseline : 0.802
* NNPU Raw : 0.787
* NNPU Transform : 0.66


# Experiment 3
NClusters = 1
Dim = 2
aucpn_range=  [.8,1.]
NBags = 50
NPos = 50
NUnlabeled = 100

* True Posterior : 0.884
* NNPU Transform : 0.864
* NNPU Raw : 0.86
* NNPU True Rank : 0.695
* NNPU Estimated Rank : 0.682
* Estimated Posterior : 0.58 (posterior auc 0.836)


# Experiment 4
NClusters = 3
Dim = 2
aucpn_range=  [.8,1.]
NBags = 1
NPos = 100
NUnlabeled = 1000

* True Posterior : 0.775
* Estimated Posterior : 0.719 posterior cluster aucs : (.72, 86, .68)
* Baseline : 0.698
* NNPU Raw : 0.603
* NNPU Estimated Rank : 0.33
* NNPU True Rank : 0.3255
* NNPU Transform : 0.309


# Experiment 5
NClusters = 3
Dim = 2
aucpn_range=  [.8,1.]
NBags = 10
NPos = 100
NUnlabeled = 1000

* True Rank : 0.818
* True Posterior : 0.77
* NNPU Raw : 0.757
* NNPU Transform :  0.711
* Estimated Posterior : 0.69 (0.69, .87, .77) 
* Baseline : 0.656
* Estimated Rank : 0.56


# Experiment 6
NClusters = 3
Dim = 2
aucpn_range=  [.8,1.]
NBags = 100
NPos = 100
NUnlabeled = 1000

* True Posterior : 0.907
* True Rank : 0.89
* NNPU Raw : 0.798
* NNPU Transform : 0.710
* baseline : 0.69
* Est. Posterior : 0.61
    * posteriors auc 0.878
    * posteriors auc 0.860
    * posteriors auc 0.833
* Est Rank : 0.601

$\frac{n_p + \alpha n_u}{n_p + n_u}$

# Experiments for Table

# Experiment 0
NClusters = 3
Dim = 64
aucpn_range=  [.8,1.]
NBags = 1000
NPos = 10
NUnlabeled = 25

* True Posterior : 0.876
* True Rank : 0.79
* NNPU Transform : 0.704
* NNPU Raw : 0.628
* Estimated Rank : 0.58
* Estimated Posterior : 0.573
    * posterior auc  0.822
    * posterior auc  0.855
    * posterior auc  0.849
    * 0.83
* baseline : 0.52 (700 bags seems to have stabilized)


# Experiment 1
NClusters = 3
Dim = 32
aucpn_range=  [.8,1.]
NBags = 1000
NPos = 10
NUnlabeled = 25

* True Posterior : 0.83
* True Rank : 0.76
* NNPU Raw : 0.736
* NNPU Transform : 0.685
* Est Rank : 0.587
* Estimated Poster : 0.542
    * Cluster AUCs : .83, .80, .77
    * Merged AUC calculated directly : 0.798
* baseline : 0.52

# Experiment 2
NClusters = 3
Dim = 16
aucpn_range=  [.8,1.]
NBags = 1000
NPos = 10
NUnlabeled = 25

* True Posterior : 0.785
* NNPU True Rank : 0.73
* NNPU Transform : 0.65
* Estimated Posterior : 0.63
    * .78, .79, .74
* NNPU Raw : 0.595
* NNPU Estimated Rank : 0.54
* baseline : 0.527

# Experiment 3

NClusters = 3
Dim = 8
aucpn_range=  [.8,1.]
NBags = 1000
NPos = 10
NUnlabeled = 25

* True Posterior : 0.755
* True Rank : 0.714
* NNPU Transform : 0.642
* Estimated Posterior : 0.637
* NNPU Raw : 0.6355
* Estimated Rank : 0.572
* baseline : 